In [517]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import OneHotEncoder


In [518]:
df_train_raw = pd.read_csv('downloaded/train.csv')
pd.options.display.max_rows = None

df['Embarked'] = df['Embarked'].fillna(3333)

In [519]:
df.index

RangeIndex(start=0, stop=891, step=1)

In [520]:
# df.dropna() drops the percentage correct to .764

# Do some preprocessing here
df_train_raw = pd.read_csv('downloaded/train.csv')
def do_preprocessing(df):
    sex_mapping = {
        'female': 0,
        'male': 1,
    }
    df['Sex'] = df['Sex'].map(sex_mapping)
    
    # for missing Fare put in the mean
    imr = Imputer(missing_values='NaN', strategy='mean', axis=0)
    print(df['Fare'].values.dtype)
    imr = imr.fit(df['Fare'].values.reshape(-1, 1))
    imputed_data = imr.transform(df['Fare'].values.reshape(-1,1))
    df['Fare'] = pd.DataFrame(imputed_data)
    
    # TODO turn Embarked Q, S, C into binary columns
    df['Embarked'] = df['Embarked'].fillna('Missing')
    df['Embarked'] = df['Embarked'].map({    
        'Q':0,
        'S':1,
        'C':2,
        'Missing':3
    })

    
    ohe = OneHotEncoder(categorical_features=[0], n_values = 4)
    x = ohe.fit_transform(df['Embarked'].values.reshape(-1, 1)).toarray()
    x = pd.DataFrame(x)
    
    x = x.rename(index=str, columns={
        0: "embarked_q", 
        1: "embarked_s",
        2: "embarked_c",
        3: "embarked_missing",
    })
    x.index = range(len(x))


    df.index = range(len(df))
    
    df = df.join(x)
    
    return df


df_train = do_preprocessing(df_train_raw)

# skipping non numeric columns and columns that have nulls in some rows
# TODO need better solution for nulls and numeric columns
# Age -> is about 20% null in the trainning data. for now i think there is lower hanging fruit
# Fare has 1 null in testing data
# eventually should probably be using of these columns 
# 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked'

# note using Embarked Made it worse for now. well, better on my trainning data nd worse on the
# submission
#training_columns = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'embarked_q', 'embarked_s', 
# 'embarked_missing', 'embarked_c']

training_columns = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare']



float64


In [521]:
y_train = df_train['Survived']

#x_train = df_train[training_columns]

x_train, x_test, y_train, y_test = train_test_split(df_train[training_columns], 
    y_train, test_size=0.3, random_state=0)

# feature scaling not needed for a decision tree but may want to add in later
#x_train.describe()


In [522]:
# let's just explore null values in this cell
for i in (['downloaded/train.csv', 'downloaded/test.csv']):
    df_tmp = pd.read_csv(i)
    print('total rows:{}'.format(len(df_tmp)))
    print(df_tmp.isnull().sum())


total rows:891
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
total rows:418
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [523]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0)
tree.fit(x_train.as_matrix(), y_train.as_matrix())
y_pred = tree.predict(x_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))
x_test.describe()

Accuracy: 0.806


,Pclass,Sex,SibSp,Parch,Fare
count,268.000000,268.000000,268.000000,268.000000,268.000000
mean,2.358209,0.630597,0.503731,0.354478,31.613603
std,0.824739,0.483546,0.950057,0.742817,52.969870
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.000000,7.895800
50%,3.000000,1.000000,0.000000,0.000000,13.000000
75%,3.000000,1.000000,1.000000,0.000000,29.268750
max,3.000000,1.000000,5.000000,5.000000,512.329200


# Last step generate the file to send to kaggle

In [524]:
df_test_raw = pd.read_csv('downloaded/test.csv')

print(df_test_raw['Embarked'].unique())
#df_test = do_preprocessing(df_test_raw)

#df_test
#x_test = df_test[training_columns]


['Q' 'S' 'C']


In [525]:
df_train_raw['Embarked']

0      1
1      2
2      1
3      1
4      1
5      0
6      1
7      1
8      1
9      2
10     1
11     1
12     1
13     1
14     1
15     1
16     0
17     1
18     1
19     2
20     1
21     1
22     0
23     1
24     1
25     1
26     2
27     1
28     0
29     1
30     2
31     2
32     0
33     1
34     2
35     1
36     2
37     1
38     1
39     2
40     1
41     1
42     2
43     2
44     0
45     1
46     0
47     0
48     2
49     1
50     1
51     1
52     2
53     1
54     2
55     1
56     1
57     2
58     1
59     1
60     2
61     3
62     1
63     1
64     2
65     2
66     1
67     1
68     1
69     1
70     1
71     1
72     1
73     2
74     1
75     1
76     1
77     1
78     1
79     1
80     1
81     1
82     0
83     1
84     1
85     1
86     1
87     1
88     1
89     1
90     1
91     1
92     1
93     1
94     1
95     1
96     2
97     2
98     1
99     1
100    1
101    1
102    1
103    1
104    1
105    1
106    1
107    1
108    1
109    0
110    1
1

In [526]:
df_test_raw = pd.read_csv('downloaded/test.csv')

df_test = do_preprocessing(df_test_raw)
x_test = df_test[training_columns]

#df_test = do_preprocessing(df_test_raw[trainning_columns])
#x_test = df_test

result = tree.predict(x_test)

# last step is output the results to a csv
df_for_file = df_test[['PassengerId',]]
df_result = pd.DataFrame(result)
df_result.columns = ['Survived']

# get id of false pandas warning
pd.options.mode.chained_assignment = None

df_for_file['Survived'] = df_result[['Survived',]]

df_for_file.to_csv("out.csv", header = ['PassengerId', 'Survived'], index=False)

float64
